In [ ]:
import sys
import numpy as np
import time
sys.path.append('./BIC_codes/')
from functions.dFC_funcs import (
    DATA_LOADER, 
    DFC_ANALYZER, 
    TR_intersection,
    SIMILARITY_ASSESSMENT
)


### Parameters 

1. Set paths to the data folders and output folder, select data type, and the sessions of the data and networks that you want to include

In [ ]:
output_root = './../../../../../RESULTs/methods_implementation/'

params_data_load = { \
    'DATA_type': 'Gordon', \
    'SESSIONs':['Rest1_LR'], \
    'networks2include':['Auditory', 'CinguloOperc', 'Default', 'DorsalAttn', 'FrontoParietal',
        'MedialParietal', 'ParietoOccip', 'SMhand', 'SMmouth',
        'Salience', 'VentralAttn', 'Visual'], \

    'data_root_gordon': './sampleDATA/', 
    'data_root_ica': None 
}

2. Set the default parameter values for dFC assessment

In [ ]:
params_methods = { \
    # Sliding Parameters
    # W is in sec
    'W': 44, 'n_overlap': 0.5, 'sw_method':'pear_corr', 'tapered_window':True, \
    # TIME_FREQ
    'TF_method':'WTC', \
    # CLUSTERING AND DHMM
    'clstr_base_measure':'SlidingWindow', \
    # HMM
    'hmm_iter': 50, 'n_hid_states': 24, \
    # State Parameters
    'n_states': 12, 'n_subj_clstrs': 20, \
    # Parallelization Parameters
    'n_jobs': 2, 'verbose': 0, 'backend': 'loky', \
    # SESSION
    'session': 'Rest1_LR', \
    # Hyper Parameters
    'normalization': True, \
    'num_subj': 2, \
    'num_select_nodes': 96, \
    'num_time_point': 1200, \
    'Fs_ratio': 1.00, \
    'noise_ratio': 0.00, \
    'num_realization': 1 \
}

3. Specify the list of methods you want to include:

In [ ]:
MEASURES_name_lst = [ 
                'SlidingWindow', 
                # 'Time-Freq', 
                # 'CAP', 
                # 'ContinuousHMM', 
                'Windowless', 
                # 'Clustering', 
                # 'DiscreteHMM' 
]

4. You may set a list of alternative values for each parameter

In [ ]:
alter_hparams = { 
            'n_states': [6], 
}

5. Set parameter values for dFC Analyzer

In [ ]:
params_dFC_analyzer = { \
    # Parallelization Parameters
    'n_jobs': None, 'verbose': 0, 'backend': 'loky' 
}

### LOAD DATA

Creating Data Loader object and loading the data as a Time Series object

In [ ]:
data_loader = DATA_LOADER(**params_data_load)
BOLD = data_loader.load()

How the info dictionary of the loaded Time Series look like:

In [ ]:
BOLD['Rest1_LR'].info

This is what the BOLD signals look like:

In [ ]:
BOLD['Rest1_LR'].visualize(
    start_time=0, end_time=1000, 
    nodes_lst=range(10), 
)

### Measures of dFC 

1. Create dFC Multi-Analysis object:

In [ ]:
dFC_analyzer = DFC_ANALYZER( 
    analysis_name='reproducibility assessment', 
    **params_dFC_analyzer 
)

2. Initialize the list of measures with the given hyperparameters

In [ ]:
MEASURES_lst = dFC_analyzer.measures_initializer( 
    MEASURES_name_lst, 
    params_methods, 
    alter_hparams 
)

There are one instance of SlidingWindow, and two instances of Window-less, one with 12 states and the other one with 6 states.

In [ ]:
print(MEASURES_lst)

### estimate FC states

Run FCS estimation for state-based methods only and collect them in MEASURES_fit_lst

In [ ]:
tic = time.time()
print('Measurement Started ...')

MEASURES_fit_lst = list()
for measure in MEASURES_lst:

    time_series = BOLD[measure.params['session']]
    if measure.is_state_based:
        measure.estimate_FCS(time_series=time_series)
    MEASURES_fit_lst.append(measure)
            
print('Measurement required %0.3f seconds.' % (time.time() - tic, ))

dFC_analyzer.set_MEASURES_fit_lst(MEASURES_fit_lst)

How info dictionary of a measure looks like (e.g. for the SlidingWindow object):

In [ ]:
MEASURES_fit_lst[0].info

### dFC assessment

Assess each subject's dFC using the fitted measure objects:

In [ ]:
tic = time.time()
print('Measurement Started ...')

print("dFCM estimation started...")

SUBJ_dFCM_dict = {}
for subj in range(dFC_analyzer.params_methods['num_subj']):
    subj_id = data_loader.SUBJECTS[subj]
    BOLD = data_loader.load(subj_id2load=subj_id)
    dFCM_dict = dFC_analyzer.subj_lvl_dFC_assess(time_series_dict=BOLD)
    SUBJ_dFCM_dict[subj_id] = dFCM_dict

print("dFCM estimation done.")

print('Measurement required %0.3f seconds.' % (time.time() - tic, ))

VISUALIZE dFC

You may filter the dFCMs you want to visualize by specifying values of parameters in param_dict:

In [ ]:
dFCM_lst = SUBJ_dFCM_dict[subj_id]['dFCM_lst']

Visualize a sample interval of assessed dFC using different measures:

In [ ]:
TRs = TR_intersection(dFCM_lst)
chosen_TRs = TRs[10:20]

for dFCM in dFCM_lst:
    print(dFCM.measure.measure_name)
    dFCM.visualize_dFC(TRs=chosen_TRs, normalize=False, fix_lim=False)

### Similarity assessment

In [ ]:
similarity_assessment = SIMILARITY_ASSESSMENT(dFCM_lst=dFCM_lst)

tic = time.time()
print('Measurement Started ...')

print("Similarity measurement started...")
SUBJ_output = similarity_assessment.run(FILTERS=dFC_analyzer.hyper_param_info, downsampling_method='default')
print("Similarity measurement done.")

print('Measurement required %0.3f seconds.' % (time.time() - tic, ))

This is what similarity results of each subject contain:

In [ ]:
print(SUBJ_output.keys())
print(SUBJ_output['default_values'].keys())

### Post Analysis

1. Create a dictionary containing RESULTS to be visualized

In [ ]:
metric = 'spearman'
filter = 'default_values'

similarity_mat = np.squeeze(SUBJ_output[filter]['all'][metric])
methods_names = [measure.measure_name for measure in SUBJ_output[filter]['measure_lst']]
RESULTS = {'session1_LR':{
                        'similarity_mat':similarity_mat,
                        'methods_names':methods_names
            }
}

2. VISUALIZE Similarity results

In [ ]:
from functions.post_analysis_funcs import visualize_sim_mat

visualize_sim_mat(RESULTS, 
                    mat_key='similarity_mat', 
                    name_lst_key='methods_names', 
                    cmap='viridis',
)